In [1]:
import socket
from time import sleep

PORT = 80
HOST = 'httpbin.org'
PATH = '/image/png'
CRLF = b'\r\n'
HTTP_PATH = f'GET {PATH} HTTP/1.1'.encode('ascii') + CRLF
HTTP_HOST = f'Host: {HOST}'.encode('ascii') + CRLF

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((HOST, PORT))
    s.sendall(HTTP_PATH)
    s.sendall(HTTP_HOST)
    s.sendall(CRLF)
    data = s.recv(1024)
    sleep(1)
    data += s.recv(9999)

len(data), data[:300], data[-10:]

(8314,
 b'HTTP/1.1 200 OK\r\nDate: Mon, 23 Mar 2020 02:05:55 GMT\r\nContent-Type: image/png\r\nContent-Length: 8090\r\nConnection: keep-alive\r\nServer: gunicorn/19.9.0\r\nAccess-Control-Allow-Origin: *\r\nAccess-Control-Allow-Credentials: true\r\n\r\n\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00d\x00\x00\x00d\x08\x02\x00\x00\x00\xff\x80\x02\x03\x00\x00\x1faIDATx\x9c\xdd}wXS\xd9\xd6\xfeJ#\xa1\x17\xc1 \x02\xa1\x08\xc1J\x19\x11E\x10\x1c\x11\x14\x10\xf8tl(\xea\xd8',
 b'\x00\x00IEND\xaeB`\x82')

In [2]:
print(data.decode('ascii', errors='backslashreplace')[:300])

HTTP/1.1 200 OK
Date: Mon, 23 Mar 2020 02:05:55 GMT
Content-Type: image/png
Content-Length: 8090
Connection: keep-alive
Server: gunicorn/19.9.0
Access-Control-Allow-Origin: *
Access-Control-Allow-Credentials: true

\x89PNG

IHDR   d   d   \xff\x80  aIDATx\x9c\xdd}wXS\xd9\xd6\xfeJ


In [3]:
with open('../../tests/assets/cat.jpg', 'rb') as fd:
    cat_photo_bytes = fd.read()

CONTENT_LENGHT = str(len(cat_photo_bytes)).encode('ascii')

CRLF = b'\r\n'
HTTP_INIT = b'HTTP/1.1 200 OK' + CRLF
HTTP_HEADERS = b'Content-Type: image/png' + CRLF + b'Content-Length: ' + CONTENT_LENGHT + CRLF
HTTP_BODY = cat_photo_bytes
HTTP_MESSAGE = HTTP_HEADERS + CRLF + HTTP_BODY

HTTP_RESPONSE = HTTP_INIT + HTTP_MESSAGE

print(HTTP_MESSAGE.decode('ascii', errors='backslashreplace')[:300])

Content-Type: image/png
Content-Length: 1225

	


 ' .)10.)-,3:J>36F7,-@WAFLNRSR2>ZaZP`JQRO\xff\xdb C&&O5-5OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO\xff\xc0  - P" \xff\xc4           	
\xff\


In [4]:
import email

msg = email.message_from_bytes(HTTP_MESSAGE)

msg

In [5]:
msg.items()

[('Content-Type', 'image/png'), ('Content-Length', '1225')]

In [6]:
msg_payload = msg.get_payload(decode=True)

len(msg_payload), type(msg_payload)

(1225, bytes)

In [7]:
msg_payload == cat_photo_bytes

True

In [8]:
# import socket

# with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
#     s.bind(('127.0.0.1', 8000))
#     s.listen(1)
#     conn, addr = s.accept()
#     print(conn, addr)
#     with conn:
#         print('Connected by', addr)
#         while True:
#             request_data = conn.recv(1024)
#             if not request_data: break
#             conn.sendall(HTTP_RESPONSE)
#             break